In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [20]:
#RDD1 registros de ventas de producto con la forma (fecha de venta, código de producto, precio de venta)
#RDD2 detalle de los productos con (código de producto,descripción del producto, categoría)

registro_ventas = [
    ('20160101', 10,100),
    ('20160202', 20,200),
    ('20160402', 15,150),
    ('20160405', 20,200),
    ('20160410', 25,200),
    ('20160415', 15,150),
    ('20160420', 40,400),
    ('20160505', 80,500),
    ('20160507', 80,500),
    ('20161005', 80,500),
    ('20161130', 80,500),
    ('20170105', 80,600),
    ('20170101', 10,100),
    ('20170202', 20,200),
    ('20170402', 15,150),
    ('20170405', 20,200),
    ('20170410', 25,250),
    ('20170415', 15,150),
    ('20170420', 40,400),
    ('20170505', 80,800),
    ('20170511', 80,800),
    ('20170517', 80,800),
    ('20170522', 80,800),
    ('20170525', 80,800),
    ('20171205', 80,800),
]

registro_detalle_prod = [
    ( 10,'PackGalletitas','comestibles_almacen'),
    ( 15,'AlgodonFamiliar','higiene_limpieza'),
    ( 20,'PapelHigienico','higiene_limpieza'),
    ( 25,'PastaAvellanas','comestibles_almacen'),
    ( 40,'Milanesas','comestibles_carniceria'),
    ( 80,'PolvoLavarropasFamiliar','higiene_limpieza')
]

In [21]:
data = sc.parallelize(registro_ventas);
data0 = sc.parallelize(registro_detalle_prod)

In [22]:
#Genero un RDD con codigo de producto y un 1 para cada venta (clave,valor)
data1=data.map(lambda x: (x[1],1))
data1.collect()

[(10, 1),
 (20, 1),
 (15, 1),
 (20, 1),
 (25, 1),
 (15, 1),
 (40, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (10, 1),
 (20, 1),
 (15, 1),
 (20, 1),
 (25, 1),
 (15, 1),
 (40, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1)]

In [23]:
#data1 es un RDD
type(data1)

pyspark.rdd.PipelinedRDD

In [24]:
#Una vez que a data1 le hago un collect() obtengo una lista
data3=data1.collect()
type(data3)

list

In [25]:
#Sumo todos los 1`s para cada codigo de producto,obtengo el mas vendido con un reduceByKey
#y luego con un reduce busco el valor mayor obteniendo el producto mas vendido
data2=data1.reduceByKey(lambda x, y: (x+y)).map(lambda x: (x[1],x[0])).reduce(lambda x ,y : x if x > y else y )
data2=(data2[1],data2[0])
data2

(80, 11)

In [34]:
#Version mas breve para obtener el producto mas vendido
data2b=data1.reduceByKey(lambda x, y: (x+y)).reduce(lambda x ,y : x if x[1] > y[1] else y )
data2b

(80, 11)

In [26]:
#Filtro el RDD con detalles de productos para obtener cual es el prod. mas vendido
data0.filter(lambda x:(x[0] == data2[0])).collect()
#esto me da un RDD

[(80, 'PolvoLavarropasFamiliar', 'higiene_limpieza')]

In [27]:
#Rta.A el producto mas vendido 
dataProd=data0.filter(lambda x:(x[0] == data2[0])).collect()
dataProd[0][1]

'PolvoLavarropasFamiliar'

In [28]:
data0b=data0.map(lambda x: (x[0],x[2]))
data0b.collect()

[(10, 'comestibles_almacen'),
 (15, 'higiene_limpieza'),
 (20, 'higiene_limpieza'),
 (25, 'comestibles_almacen'),
 (40, 'comestibles_carniceria'),
 (80, 'higiene_limpieza')]

In [29]:
data1B=data1.join(data0b)
print(data1B)

PythonRDD[58] at RDD at PythonRDD.scala:52


In [30]:
data1B.collect()

[(40, (1, 'comestibles_carniceria')),
 (40, (1, 'comestibles_carniceria')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (80, (1, 'higiene_limpieza')),
 (25, (1, 'comestibles_almacen')),
 (25, (1, 'comestibles_almacen')),
 (10, (1, 'comestibles_almacen')),
 (10, (1, 'comestibles_almacen')),
 (20, (1, 'higiene_limpieza')),
 (20, (1, 'higiene_limpieza')),
 (20, (1, 'higiene_limpieza')),
 (20, (1, 'higiene_limpieza')),
 (15, (1, 'higiene_limpieza')),
 (15, (1, 'higiene_limpieza')),
 (15, (1, 'higiene_limpieza')),
 (15, (1, 'higiene_limpieza'))]

In [31]:
data1B2=data1B.map(lambda x: (x[1][1],x[1][0]))
data1B2.collect()

[('comestibles_carniceria', 1),
 ('comestibles_carniceria', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('comestibles_almacen', 1),
 ('comestibles_almacen', 1),
 ('comestibles_almacen', 1),
 ('comestibles_almacen', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1),
 ('higiene_limpieza', 1)]

In [35]:
data1B3=data1B2.reduceByKey(lambda x, y: (x+y)).reduce(lambda x ,y : x if x[1] > y[1] else y )
data1B3

('higiene_limpieza', 19)

In [33]:
#Rta.B la categoria mas vendida
data1B3[0]

'higiene_limpieza'

In [150]:
dataTop5=data1.reduceByKey(lambda x, y: (x+y)).collect()
print(dataTop5)

[(20, 4), (40, 2), (80, 10), (25, 2), (10, 2), (15, 4)]


In [159]:
def getValue(item):   
    return item[1]
resultadoFinal=sorted(dataTop5, key=getValue)
resultadoFinal.reverse()
resultadoFinal
# O sino con resultadoFinal[-5::] obtengo un top 5 en orden ascendente ,habria que invertir la lista con reverse() notar
# que lleva -5 y dos "::" el 2do ":" o ":" de la derecha es para tomar todos los items desde el ultimo y el -5 para llegar
# hasta esa posicion


[(80, 10), (15, 4), (20, 4), (10, 2), (25, 2), (40, 2)]

In [160]:
data0c = sc.parallelize(resultadoFinal)

In [161]:
#Rta.C RDD top 5 con codigo de producto, cantidad de ventas y descripcion
dataTop5b=data0c.rightOuterJoin(data0).collect()
dataTop5b

[(80, (10, 'PolvoLavarropasFamiliar')),
 (40, (2, 'Milanesas')),
 (25, (2, 'PastaAvellanas')),
 (10, (2, 'PackGalletitas')),
 (20, (4, 'PapelHigienico')),
 (15, (4, 'AlgodonFamiliar'))]

In [163]:
#ordeno el RDD con el top 5
def getValue2(item):   
    return item[1][0]
resultadoFinal2=sorted(dataTop5b, key=getValue2)
resultadoFinal2.reverse()
resultadoFinal2

[(80, (10, 'PolvoLavarropasFamiliar')),
 (15, (4, 'AlgodonFamiliar')),
 (20, (4, 'PapelHigienico')),
 (10, (2, 'PackGalletitas')),
 (25, (2, 'PastaAvellanas')),
 (40, (2, 'Milanesas'))]

In [10]:
#Busco los productos que tuvieron 50 ventas como minimo durante el ultimo año
#A fines practicos tomo aquellos que tuvieron mas de 2 ventas el ultimo año
dataD=data.filter(lambda x: (x[0] >='20170101'))

In [63]:
dataD.collect()

[('20170105', 80, 600),
 ('20170101', 10, 100),
 ('20170202', 20, 200),
 ('20170402', 15, 150),
 ('20170405', 20, 200),
 ('20170410', 25, 250),
 ('20170415', 15, 150),
 ('20170420', 40, 400),
 ('20170505', 80, 800),
 ('20170511', 80, 800),
 ('20170517', 80, 800),
 ('20170522', 80, 800),
 ('20170525', 80, 800),
 ('20171205', 80, 800)]

In [102]:
#Ordeno los campos del RDD para dejar como clave el codigo de producto
dataD3=dataD.map(lambda x: (x[1],(x[2],x[0])))
dataD3.collect()

[(80, (600, '20170105')),
 (10, (100, '20170101')),
 (20, (200, '20170202')),
 (15, (150, '20170402')),
 (20, (200, '20170405')),
 (25, (250, '20170410')),
 (15, (150, '20170415')),
 (40, (400, '20170420')),
 (80, (800, '20170505')),
 (80, (800, '20170511')),
 (80, (800, '20170517')),
 (80, (800, '20170522')),
 (80, (800, '20170525')),
 (80, (800, '20171205'))]

In [82]:
#Armo un RDD para obtener cuantos productos por codigo se vendieron y luego filtrar por cantidad
#vendida durante el ultimo año (se pide 50 tomo 2 por comodidad)
dataD1=dataD.map(lambda x: (x[1],1))
dataD1.collect()

[(80, 1),
 (10, 1),
 (20, 1),
 (15, 1),
 (20, 1),
 (25, 1),
 (15, 1),
 (40, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1),
 (80, 1)]

In [73]:
type(dataD1)

pyspark.rdd.PipelinedRDD

In [83]:
#Me quedo con solo aquellos que tuvieron mas de 2 ventas el ultimo año (2017)
dataD2=dataD1.reduceByKey(lambda x, y : (x+y)).filter(lambda x:(x[1] >= 2))

In [84]:
dataD2.collect()

[(80, 7), (20, 2), (15, 2)]

In [94]:
#Lista de productos con mas de 2 unidades vendidas el ultimo año
dataD2b=dataD2.map(lambda x: (x[0]))
lista=dataD2b.collect()
lista

[80, 20, 15]

In [105]:
#Uno los RDD's con los productos correspondientes al ultimo año(2017) y aquellos productos que se vendieron
#en mas de 50 unidades (2 en este caso a fines practicos)
dataD4=dataD3.rightOuterJoin(dataD2)

In [106]:
dataD4.collect()

[(80, ((600, '20170105'), 7)),
 (80, ((800, '20170505'), 7)),
 (80, ((800, '20170511'), 7)),
 (80, ((800, '20170517'), 7)),
 (80, ((800, '20170522'), 7)),
 (80, ((800, '20170525'), 7)),
 (80, ((800, '20171205'), 7)),
 (20, ((200, '20170202'), 2)),
 (20, ((200, '20170405'), 2)),
 (15, ((150, '20170402'), 2)),
 (15, ((150, '20170415'), 2))]

In [120]:
#Arreglo el RDD quitando las tuplas internas
listaD=dataD4.map(lambda x: (x[0],x[1][1],x[1][0][0],x[1][0][1]))
listaD.collect()

[(80, 7, 600, '20170105'),
 (80, 7, 800, '20170505'),
 (80, 7, 800, '20170511'),
 (80, 7, 800, '20170517'),
 (80, 7, 800, '20170522'),
 (80, 7, 800, '20170525'),
 (80, 7, 800, '20171205'),
 (20, 2, 200, '20170202'),
 (20, 2, 200, '20170405'),
 (15, 2, 150, '20170402'),
 (15, 2, 150, '20170415')]

In [119]:
#Una forma de obtener el primer registro de venta durante el ultimo año y el ultimo seria asi:
prueba=[listaD[0],listaD[listaD[0][1]-1]]
prueba

[(80, 7, 600, '20170105'), (80, 7, 800, '20171205')]

In [125]:
#Otra forma seria la siguiente:
#Me quedo con el primer producto de la lista
#Idem para los demas productos de la lista obtenida con los productos con mas de 2 ventas durante el ultimo año
listaD.filter(lambda x:x[0]==lista[0]).collect()

[(80, 7, 600, '20170105'),
 (80, 7, 800, '20170505'),
 (80, 7, 800, '20170511'),
 (80, 7, 800, '20170517'),
 (80, 7, 800, '20170522'),
 (80, 7, 800, '20170525'),
 (80, 7, 800, '20171205')]

In [126]:
#Filtro la lista obteniendo el primer registo correspondiente al ultimo año
prod1=listaD.filter(lambda x:x[0]==lista[0]).first()
prod1

(80, 7, 600, '20170105')

In [135]:
type(prod1)

tuple

In [132]:
#Filtro la lista obteniendo el ultimo registo correspondiente al ultimo año
prod1f=listaD.filter(lambda x:x[0]==lista[0]).top(1)
prod1f

[(80, 7, 800, '20171205')]

In [138]:
type(prod1f)

list

In [139]:
#Obtengo la variacion de precio durante el ultimo año,restando los registros
variacionProd1=prod1f[0][2]-prod1[2]
variacionProd1

200

In [ ]:
#Codigos de pruebas:

In [123]:
prod2=listaD.filter(lambda x:x[0]==lista[1]).collect()

In [ ]:
data0.map(lambda x: (x[0]==resultadoFinal[0][0],resultadoFinal[0][1],x[1],x[2])).collect()

In [94]:
type(resultadoFinal)

list

In [97]:
data1.reduceByKey(lambda x, y: (x+y)).takeOrdered(5)

[(10, 2), (15, 4), (20, 4), (25, 2), (40, 2)]

In [106]:
dataB=data.map(lambda x: (x[1],x[2]))
dataB.collect()

[(10, 100),
 (20, 200),
 (15, 150),
 (20, 200),
 (25, 200),
 (15, 150),
 (40, 400),
 (80, 500),
 (80, 500),
 (80, 500),
 (80, 500),
 (10, 100),
 (20, 200),
 (15, 150),
 (20, 200),
 (25, 250),
 (15, 150),
 (40, 400),
 (80, 800),
 (80, 800),
 (80, 800),
 (80, 800),
 (80, 800),
 (80, 800)]

In [109]:
dataB.fullOuterJoin(data0).collect()

[(40, (400, 'Milanesas')),
 (40, (400, 'Milanesas')),
 (80, (500, 'PolvoLavarropasFamiliar')),
 (80, (500, 'PolvoLavarropasFamiliar')),
 (80, (500, 'PolvoLavarropasFamiliar')),
 (80, (500, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (80, (800, 'PolvoLavarropasFamiliar')),
 (25, (200, 'PastaAvellanas')),
 (25, (250, 'PastaAvellanas')),
 (10, (100, 'PackGalletitas')),
 (10, (100, 'PackGalletitas')),
 (20, (200, 'PapelHigienico')),
 (20, (200, 'PapelHigienico')),
 (20, (200, 'PapelHigienico')),
 (20, (200, 'PapelHigienico')),
 (15, (150, 'AlgodonFamiliar')),
 (15, (150, 'AlgodonFamiliar')),
 (15, (150, 'AlgodonFamiliar')),
 (15, (150, 'AlgodonFamiliar'))]